In [40]:
from __future__ import print_function
from __future__ import absolute_import

import cobra
import cobra.test
import numpy as np
import csv
import glob
import pickle
import pandas as pd
import math
import copy
import time
import random
import time
import sys
import matplotlib.pyplot as plt

from copy import deepcopy
from collections import defaultdict
from cobra.flux_analysis import sample
from cobra.core.solution import get_solution
from cobra.flux_analysis.sampling import OptGPSampler
from cobra.manipulation.delete import *
from cobra.medium import find_boundary_types
from cobra.flux_analysis import pfba

from warnings import warn
from itertools import chain
from optlang.symbolics import Zero
from cobra.util import solver as sutil
from cobra.core.solution import get_solution

import logging
LOGGER = logging.getLogger(__name__)

In [21]:
count = -1
all_active_list = []
for genome_id in genome_ids:
    file_name = "../metabolic_output_V3/%s.data" % (genome_id)
    total_dataset_dict = pickle.load(open(file_name, "rb"))
    count += 1
    sys.stdout.write(str(count)+'_')
    ID = str(genome_id + ':' + str(0)+'.'+str(29))
    active = total_dataset_dict[ID]['active_rxns']
    active = [str(i) for i in list(active.index)]
    all_active_list = all_active_list + active
all_active_set = set(all_active_list)

0_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15_16_17_18_19_20_21_22_23_24_25_26_27_28_29_30_31_32_33_34_35_36_37_38_39_40_41_42_43_44_45_46_47_48_49_50_51_52_53_54_55_56_57_58_59_60_61_62_63_64_65_66_67_68_69_70_71_72_73_74_75_76_77_78_79_80_81_82_83_84_85_86_87_88_89_90_91_92_93_94_95_96_97_98_99_100_101_102_103_104_105_106_107_108_109_110_111_112_113_114_115_116_117_118_119_120_121_122_123_124_125_126_127_128_129_130_131_132_133_134_135_136_137_138_139_140_141_142_143_144_145_146_147_148_149_150_151_152_153_154_155_156_157_158_159_160_161_162_163_164_165_166_167_168_169_170_171_172_173_174_175_176_177_178_179_180_181_182_183_184_185_186_187_188_189_190_191_192_193_194_195_196_197_198_199_200_201_202_203_204_205_206_207_208_209_210_211_212_213_214_215_216_217_218_219_220_221_222_223_224_225_226_227_228_229_230_231_232_233_234_235_236_237_238_239_240_241_242_243_244_245_246_247_248_249_250_251_252_253_254_255_256_257_258_259_260_261_262_263_264_265_266_267_268_269_270_271_272_273_274_275_276_27

AttributeError: 'list' object has no attribute 'appned'

In [22]:
output = []
for rxn in all_active_set:
    rxn_count = all_active_list.count(rxn)
    rxn_percent = 100*rxn_count/len(genome_ids)
    output.append([rxn,rxn_percent])

In [25]:
pickle.dump(output,open('../Data/rxn_percents_d_lactate.data','wb'))

In [41]:
# Actually prune all unused metabolites and reactions (innate function does not work)
def removeUnused(model):
    removed_cpd = set()
    removed_rxn = set()
    unused_current_cpd = 1
    unused_current_rxn = 1
    
    while unused_current_cpd != 0 or unused_current_rxn != 0:
        unused_cpd = prune_unused_metabolites(model)
        removed_cpd |= set(unused_cpd)
        unused_rxn = prune_unused_reactions(model)
        removed_rxn |= set(unused_rxn)
        
        unused_current_cpd = len(unused_cpd)
        unused_current_rxn = len(unused_rxn)
    
    print('Pruned ' + str(len(removed_cpd)) + ' metabolites from model')
    print('Pruned ' + str(len(removed_rxn)) + ' reactions from model')
        
    return(list(removed_cpd), list(removed_rxn))

In [72]:
universal = cobra.io.load_json_model("../Data/GramPosUni.json")
for met in universal.metabolites:
    if (met.id.endswith('_c')):
        universal.add_boundary(met, type='demand')

# Create exchange reaction and add to model
for rxn in all_active_set:
    if rxn.startswith('EX_'):
        met = rxn.replace('EX_',"")
#         print(met)
        new_exchange = cobra.Reaction(rxn)
        new_exchange.name = met + ' exchange'
        met_obj = universal.metabolites.get_by_id(met)
        new_exchange.add_metabolites({met_obj:-1})
        new_exchange.lower_bound = -1000.
        new_exchange.upper_bound = 1000.
        universal.add_reaction(new_exchange)
        universal.repair()

rxns_to_remove = set([rxn.id for rxn in universal.reactions]).difference(all_active_set)

print(len([rxn.id for rxn in universal.reactions]))
print(len(all_active_set)+len(rxns_to_remove))

for rxn in rxns_to_remove:
    universal.reactions.get_by_id(rxn).remove_from_model(remove_orphans = True)
unused_c, unused_r = removeUnused(universal)

universal_new = copy.deepcopy(universal)

met_list = [met.id for met in universal_new.metabolites]
for met in met_list:
    if met.endswith('_c'):
        met_name = str(universal.metabolites.get_by_id(met).name)+'_c'
        met_name = met_name.replace(" ","_")
    elif met.endswith('_e'):
        met_name = str(universal.metabolites.get_by_id(met).name)+'_e'
        met_name = met_name.replace(" ","_")
    universal_new.metabolites.get_by_id(met).id = met_name

model_file_name = '../Data/superset_d_lactate.xml'
cobra.io.write_sbml_model(universal_new, model_file_name)

15013
15013
Pruned 0 metabolites from model
Pruned 0 reactions from model


In [58]:
print(len(all_active_set))
print(len(rxns_to_remove))

109
14904
